<a href="https://colab.research.google.com/github/AlanAmaro13/Transistor_Characterization/blob/main/Transistor_Characterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transistor / Amplifier Characterization

Hey! This code provides a set of tools to characterize Amplifiers in emisor or base common configuration, this is done by providing it a set of parameters:

*   VCC - DC Supply
*   RE - Emisor Resistor
*   R1 - First resistor in the voltaje divisor
*   R2 - Second resistor in the voltaje divisor
*   R3 - Third resistor in the voltaje divisor
*   RC - Collector resistor
*   BETA - Beta/HF value
*   RL - Carge resistor
*   CC - Collector capacitance
*   CE - Emisor Capacitance
*   CB - Base Capacitance

Also it's able to resolve for a voltaje divisor using 1, 2 or 3 resistors. The output corresponds to:

*   VBB
*   RB
*   VBE
*   IB
*   IC
*   VCE
*   HFE
*   HIE
*   GV
*   GI
*   ZI
*   ZO
*   FCB
*   FCE
*   FCC
*   F_min - Minimum amplification frequency for a stable voltage
*   VI_Max

This code implements this equations:

\begin{equation}
\begin{split}
    V_{BB} &= \frac{V_{CC} R_{2} }{R_{1} + R_{2}} \\\\
    R_{B} &= \frac{R_{2}R_{1}}{R_{2} + R_{1}} \\\\
    V_{BE} &= 0.7 V \\ \\
    i_{B} &= \frac{V_{BB} - V_{BE}}{ R_{B} + R_{E}(\beta + 1) } \\\\
    i_{c} &= \beta i_{B} \\ \\
    V_{CE} &= V_{CC} - [ R_{c} \beta + R_{E}(\beta + 1) ]i_{B} \\\\
    h_{fe} &= \beta \\ \\
    h_{ie} &= \frac{0.026}{i_{B}} \\
    G_{V} &= \frac{h_{fe}}{h_{ie}}(R_{C}||R_{L}) \\ \\
    G_{I} &= h_{fe} (\frac{R_{B}}{R_{B} + h_{ie}})(\frac{R_{C}}{R_{C} + R_{L}}) \\ \\
    z_{i} &= R_{B}||h_{ie} \\ \\
    z_{o} &= R_{C} \\\\
    f_{CB}& = \frac{1}{2\pi R_{B}C_{B}} \\ \\
    f_{CE} &= \frac{1}{2\pi R_{E}C_{E}} \\\\
    f_{CC} &= \frac{1}{2\pi R_{C}C_{C}} \\\\
    |V_{i - Max}| &= \frac{V_{CC}}{G_{V}}
\end{split}
\end{equation}
And for base common transistor, the DC analysis is analogous, and the AC expressions corresponds to:

\begin{equation}
\begin{split}
    h_{fb} &=\alpha = \frac{\beta}{\beta + 1} \\ \\
    h_{ib} &= \frac{0.026 }{(\beta+1)i_{B}} \\ \\
    G_{V} &= \frac{ h_{fb} }{ h_{ib} } (R_{c}||R_{L}) \\ \\
    G_{I} &= h_{fb}(\frac{R_{C}}{R_{C} + R_{L} })(\frac{R_{E}}{R_{E} + h_{ib}}) \\ \\
    z_{i} &= R_{E}||h_{ib} \\ \\
    z_{o} &= R_{C}
\end{split}
\end{equation}







In [ ]:
! pip install prettytable

In [2]:
# Libraries
from math import pi
from prettytable import PrettyTable
table = PrettyTable()

# Transisor configuration:
config = str(input("What's the main configuration? \n Emisor o base common? [e, b]: "))
r  = int(input("How many resistor do you have in the voltage divisor? [1,2,3]: "))

# Resistor divisors
r1 = 68.4e3
r2 = 11.85e3
r3 = 10.0e3

# VCC and HFE
vcc = 12
beta = 213

# Transistor resistors
re = 670.0
rc = 3.864e3

# Charge resistor
rl = 4.721e3

# Capacitor values
cb = 63.7e-6
ce = 112.7e-6
cc = 63.7e-6


def rb_vbb_calculator(r):
  if r == 1:
    rb = r1
    vbb = vcc

  elif r==2:
    rb = (r1*r2) / (r1+r2)
    vbb =  (vcc*r2)/(r1 + r2)

  elif r==3:
    rb = ((r1 *r2) / (r1 + r2)) + r3
    vbb =  vcc * (r3 / (rb+r3))

  return [rb, vbb]

rb = rb_vbb_calculator(r)
rb = rb[0]


value_initial = [vcc, beta, r1, r2, r3, rb, re, rc, rl,
                 cb, ce, cc]

names = ["VCC", "BETA", "R1", "R2", "R3", "RB", "RE", "RC", "RL",
         "CB", "CE", "CC", "VBB", "VBE", "IB", "IC", "VCE",
         "HF", "HI", "GV", "GI", "ZI", "ZO", "FCB", "FCE",
         "FCC", "F_MIN", "VI-MAX"]



def amplificador_emisor_comun(vcc, beta, rb, re, rc, rl, cb, ce, cc):
  vbe = 0.7

  vbb = rb_vbb_calculator(r)
  vbb = vbb[1]

  ib = (vbb - vbe) / ( rb + re*(beta+1)   )

  ic = beta * ib

  vce = vcc - ( (rc*beta) + re*(beta+1)  )*ib

  hfe = beta

  hie = 0.026/ib

  gv = (hfe/hie) * ( (rc*rl)/(rc+rl) )

  gi = hfe*( rb / (rb + hie) )*(rc/(rc + rl))

  zi = ((rb*hie) / (rb + hie) )

  zo = rc

  fcb = 1/ (2*pi*rb*cb )
  fce = 1/ (2*pi*re*ce )
  fcc = 1/ (2*pi*rc*cc )

  f_list = [fcb, fce, fcc]
  f_min = max(f_list)

  vi_max = vcc/gv

  r_list = [vbb, vbe, ib, ic, vce, hfe, hie, gv, gi,
            zi, zo, fcb, fce, fcc, f_min , vi_max]

  return r_list

# ----------------------------------------------------
def amplificador_base_comun(vcc, beta, rb, re, rc, rl, cb, ce, cc):
  vbe = 0.7

  vbb = rb_vbb_calculator(r)
  vbb = vbb[1]

  ib = (vbb - vbe) / ( rb + re*(beta+1)   )

  ic = beta * ib

  vce = vcc - ( (rc*beta) + re*(beta+1)  )*ib

  hfb = (beta / (beta + 1))

  hib = 0.026/ ((beta+1)*ib)

  gv = (hfb/hib) * ( (rc*rl)/(rc+rl) )

  gi = hfb*( rc / (rc + rl) )*(re/(re+ hib))

  zi = ((re*hib) / (re + hib) )

  zo = rc

  fcb = 1/ (2*pi*rb*cb )
  fce = 1/ (2*pi*re*ce )
  fcc = 1/ (2*pi*rc*cc )

  f_list = [fcb, fce, fcc]
  f_min = max(f_list)

  vi_max = vcc/gv

  r_list = [vbb, vbe, ib, ic, vce, hfb, hib, gv, gi,
            zi, zo, fcb, fce, fcc, f_min , vi_max]

  return r_list
# ----------------------------------------------------


if config == "e":
  final_values = amplificador_emisor_comun(vcc, beta, rb, re, rc, rl, cb, ce, cc)
elif config == "b":
  final_values = amplificador_base_comun(vcc, beta, rb, re, rc, rl, cb, ce, cc)

else:
  print("No configuration selected")

final_list = value_initial + final_values

print("\n Input values corresponds to: \n\n")
table.field_names = ["Quantity", "Measure"]

for i in range(len(names)):
  table.add_row([ "{0}".format(names[i])  ,  "{0:1.3E}".format(final_list[i]) ], divider = True)

print(table)


What's the main configuration? 
 Emisor o base common? [e, b]: e
How many resistor do you have in the voltage divisor? [1,2,3]: 3

 Input values corresponds to: 


+----------+------------+
| Quantity |  Measure   |
+----------+------------+
|   VCC    | 1.200E+01  |
+----------+------------+
|   BETA   | 2.130E+02  |
+----------+------------+
|    R1    | 6.840E+04  |
+----------+------------+
|    R2    | 1.185E+04  |
+----------+------------+
|    R3    | 1.000E+04  |
+----------+------------+
|    RB    | 2.010E+04  |
+----------+------------+
|    RE    | 6.700E+02  |
+----------+------------+
|    RC    | 3.864E+03  |
+----------+------------+
|    RL    | 4.721E+03  |
+----------+------------+
|    CB    | 6.370E-05  |
+----------+------------+
|    CE    | 1.127E-04  |
+----------+------------+
|    CC    | 6.370E-05  |
+----------+------------+
|   VBB    | 3.987E+00  |
+----------+------------+
|   VBE    | 7.000E-01  |
+----------+------------+
|    IB    | 2.010E-05  |
+---